In [1]:
import numpy as np
from collections import OrderedDict
import copy

In [2]:
mnist = np.loadtxt('mnist.csv', delimiter=',')

In [3]:
def train_test_split(csv_dataset):  # i는 0~100 중의 하나의 수로 train_set의 비율을 나타낸다. ex) 70 => train_set 70% test_set 30%
    # 코드 작성
    train_X = csv_dataset[0:8000, 1:785]/255
    train_T = csv_dataset[0:8000, 0:1]
    test_X = csv_dataset[8000:10000, 1:785]/255
    test_T = csv_dataset[8000:10000, 0:1]
    return train_X, train_T, test_X, test_T
'''
[10000,785]를 조건에 맞게 4가지로 분리 train_X[8000,784](8000개 이미지 배열), train_T[8000,1](8000개 정답 배열), 
test_X[2000,784](2000개 이미지 배열), test_T [2000,1](2000개 정답 배열) 
학습시킬 이미지의 값이 0~255사이의 픽셀값이므로 256으로 나눠줌 .4가지 리턴
'''

'\n[10000,785]를 조건에 맞게 4가지로 분리 train_X[8000,784](8000개 이미지 배열), train_T[8000,1](8000개 정답 배열), \ntest_X[2000,784](2000개 이미지 배열), test_T [2000,1](2000개 정답 배열) \n학습시킬 이미지의 값이 0~255사이의 픽셀값이므로 256으로 나눠줌 .4가지 리턴\n'

In [4]:
def one_hot_encoding(T):  # T is data의 label
    # 코드 작성
    one_hot_label=np.zeros([T.shape[0], 10])

    for i in range(T.shape[0]):
        if T[i] == 0:
            one_hot_label[i][0] = 1
        elif T[i] == 1:
            one_hot_label[i][1] = 1
        elif T[i] == 2:
            one_hot_label[i][2] = 1
        elif T[i] == 3:
            one_hot_label[i][3] = 1
        elif T[i] == 4:
            one_hot_label[i][4] = 1
        elif T[i] == 5:
            one_hot_label[i][5] = 1
        elif T[i] == 6:
            one_hot_label[i][6] = 1
        elif T[i] == 7:
            one_hot_label[i][7] = 1
        elif T[i] == 8:
            one_hot_label[i][8] = 1
        elif T[i] == 9:
            one_hot_label[i][9] = 1
    return one_hot_label
'''
one_hot_encoding가 안된 정답[N,1]의 배열을 one_hot_encoding모양인[N,10]으로 변환하여 리턴해주는 함수
ex => [3] => [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
'''

'\none_hot_encoding가 안된 정답[N,1]의 배열을 one_hot_encoding모양인[N,10]으로 변환하여 리턴해주는 함수\nex => [3] => [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]\n'

In [5]:
def Softmax(ScoreMatrix):  # 제공.

    if ScoreMatrix.ndim == 2:
        temp = ScoreMatrix
        temp = temp - np.max(temp, axis=1, keepdims=True)
        y_predict = np.exp(temp) / np.sum(np.exp(temp), axis=1, keepdims=True)
        return y_predict
    temp = ScoreMatrix - np.max(ScoreMatrix, axis=0)
    expX = np.exp(temp)
    y_predict = expX / np.sum(expX)
    return y_predict



In [6]:
def setParam_He(neuronlist):
    np.random.seed(1)  # seed값 고정을 통해 input이 같으면 언제나 같은 Weight와 bias를 출력하기 위한 함수
    # 코드 작성
    W1 = np.random.randn(neournlist[0],neournlist[1])/np.sqrt(neournlist[0]/2)
    W2 = np.random.randn(neournlist[1],neournlist[2])/np.sqrt(neournlist[1]/2)
    W3 = np.random.randn(neournlist[2],neournlist[3])/np.sqrt(neournlist[2]/2)
    b1 = np.zeros(neournlist[1])
    b2 = np.zeros(neournlist[2])
    b3 = np.zeros(neournlist[3])
    return W1, W2, W3, b1, b2, b3
'''
각 linearLayer층에서 필요한 뉴런의 배열을 받아와서 
W1= [784,60] W2= [60,30] W3=[30,10] 형태로 카이밍 히의 Weight초기값 사용
b1= [60,] b2=[30,] b3=[10,] 형태로 모두 0으로 초기화
'''

'\n각 linearLayer층에서 필요한 뉴런의 배열을 받아와서 \nW1= [784,60] W2= [60,30] W3=[30,10] 형태로 카이밍 히의 Weight초기값 사용\nb1= [60,] b2=[30,] b3=[10,] 형태로 모두 0으로 초기화\n'

In [7]:
class linearLayer:
    def __init__(self, W, b):
        # backward에 필요한 X, W, b 값 저장 + dW, db값 받아오기

        self.X = None
        self.W = W
        self.b = b
        self.dW = None
        self.db = None

    def forward(self, x):
        self.X = x
        Z = np.dot(x, self.W) + self.b
        # 내적연산을 통한 Z값 계산
        return Z
    '''
    입력값 x를 받아와 W와 dot연산을 하고 그 배열에 b를 더해준다.
    그 계산한 값 Z를 반환한다.
    '''

    def backward(self, dZ):
        # 백워드 함수
        self.dW =  np.dot(self.X.T, dZ)
        self.db = np.sum(dZ, axis=0)
        dx = np.dot(dZ, self.W.T)

        return dx

    '''
    backward로 들어온 dZ를 이용해 각 단계에 알맞은 배열 모양이 되기 위해
    알맞은 연산을 한다. 
    형태가 Z=xW+b이므로
    각각 x,W,b에 관해 편미분을 하여 dx/dZ, dW/dZ, db/dZ를 구하고 
    체인룰을 이용하여 dZ/Loss와 연산한다.  
    dx값을반환하여 준다.
    '''

In [8]:
class SiLU:
    def __init__(self):
        self.Z = None  # 백워드 시 사용할 로컬 변수

    def sigmoid(self,x):
        return 1/ (1+np.exp(-x))

    def forward(self, Z): #입력값
        # 수식에 따른 forward 함수 작성
        self.Z = Z
        Activation = Z*self.sigmoid(Z)
        return Activation

    def backward(self, dActivation):
        dZ= (self.forward(self.Z) + self.sigmoid(self.Z)*(1-self.forward(self.Z)))*dActivation

        return dZ
'''
SiLU는 out = input*sigmoid(input)형태 이므로 sigmoid함수를 클래스내에 하나 정의해준다.
forward함수에서는 Z 입력값을 받아 Z*sigmoid(Z) 연산을 해주어 output인 Activation을 구하고 리턴하여준다.
첫줄에 주어진 out을 input에 대해서 편미분을 사용하면 dout/dinput=self.forward(self.Z) + self.sigmoid(self.Z)*(1-self.forward(self.Z))
이 나오므로 거기에 backward함수에서 체인룰을 이용해 dActivation과 곱셈하여 dZ를 구하고 리턴한다.
'''

'\nSiLU는 out = input*sigmoid(input)형태 이므로 sigmoid함수를 클래스내에 하나 정의해준다.\nforward함수에서는 Z 입력값을 받아 Z*sigmoid(Z) 연산을 해주어 output인 Activation을 구하고 리턴하여준다.\n첫줄에 주어진 out을 input에 대해서 편미분을 사용하면 dout/dinput=self.forward(self.Z) + self.sigmoid(self.Z)*(1-self.forward(self.Z))\n이 나오므로 거기에 backward함수에서 체인룰을 이용해 dActivation과 곱셈하여 dZ를 구하고 리턴한다.\n'

In [9]:
class SoftmaxWithLoss():  # 제공

    def __init__(self):
        self.loss = None
        self.softmaxScore = None
        self.label = None

    def forward(self, score, one_hot_label):
        batch_size = one_hot_label.shape[0]
        self.label = one_hot_label
        self.softmaxScore = Softmax(score)
        self.loss = -np.sum(self.label * np.log(self.softmaxScore + 1e-20)) / batch_size

        return self.loss

    def backward(self, dout=1):
        batch_size = self.label.shape[0]
        dx = (self.softmaxScore - self.label) / batch_size
        return dx

In [10]:
class ThreeLayerNet:

    def __init__(self, paramlist):

        W1, W2, W3, b1, b2, b3 = setParam_He(paramlist)
        self.params = {}
        self.params['W1'] = W1
        self.params['W2'] = W2
        self.params['W3'] = W3
        self.params['b1'] = b1
        self.params['b2'] = b2
        self.params['b3'] = b3

        self.layers = OrderedDict()

        self.layers['L1'] = linearLayer(self.params['W1'], self.params['b1'])
        self.layers['SiLU1'] = SiLU()
        self.layers['L2'] = linearLayer(self.params['W2'], self.params['b2'])
        self.layers['SiLU2'] = SiLU()
        self.layers['L3'] = linearLayer(self.params['W3'], self.params['b3'])

        self.lastLayer = SoftmaxWithLoss()

    def scoreFunction(self, x):

        for layer in self.layers.values():
        # 한 줄이 best
            x = layer.forward(x)
        # layer가 순차적으로 L1, SiLU1, L2, SiLU2, L3의 value에 접근하므로
        # 처음 입력받은 x(이미지 배열)로 for문의 layer의 forward()를 하면
        # L1, SiLU1, L2, SiLU2, L3 각 forward()에 접근하여 결과값을 반환한다.

        score = x
        return score
        # L3를 거쳐 나온 반환값 x는 각 이미지가 숫자10개에 대한 score이고 확률 배열과도 같다. ex[N,10]
        # 그리고 그 score를 반환한다.
    def forward(self, x, label):

        score = self.scoreFunction(x)
        return self.lastLayer.forward(score, label)

    '''
    score [N,10] N개의 이미지의 각 숫자에 대한 확률값을 scoreFunction()에서 받아와
    lastLayer의 forward의 softmax함수를 이용해 최종 Loss값을 구하고 리턴한다.
    '''

    def accuracy(self, x, label):

        score = self.scoreFunction(x)
        score_argmax = np.argmax(score, axis=1)

        if label.ndim != 1:  # label이 one_hot_encoding 된 데이터면 if문을
            label_argmax = np.argmax(label, axis=1)

        accuracy = np.sum(score_argmax == label_argmax) / int(x.shape[0])

        return accuracy

    def backpropagation(self):

        # 백워드 함수 작성 스코어펑션을 참고하세요
        x = self.lastLayer.backward()
        x = self.layers['L3'].backward(x)
        x = self.layers['SiLU2'].backward(x)
        x = self.layers['L2'].backward(x)
        x = self.layers['SiLU1'].backward(x)
        x = self.layers['L1'].backward(x)

        '''
        ScoreFunction에서는 순차적으로 L1, SiLU1, L2, SiLU2, L3에 접근하고
        backpropagation에서는 반대로 접근하면 되므로 순서만 L3, SiLU2, L2, SiLU1, L1로 설정하고 진행한다.
        backpropagation가 시행되면 내부의 각 L층에서 dW, db, dx가 결정된다.
        그리고
         밑의 grads  배열에서 그값들을 저장하고 반환한다.
        '''

        grads = {}
        grads['W1'] = self.layers['L1'].dW
        grads['b1'] = self.layers['L1'].db
        grads['W2'] = self.layers['L2'].dW
        grads['b2'] = self.layers['L2'].db
        grads['W3'] = self.layers['L3'].dW
        grads['b3'] = self.layers['L3'].db

        return grads

    def gradientdescent(self, grads, learning_rate):

        self.params['W1'] -= learning_rate * grads['W1']
        self.params['W2'] -= learning_rate * grads['W2']
        self.params['W3'] -= learning_rate * grads['W3']
        self.params['b1'] -= learning_rate * grads['b1']
        self.params['b2'] -= learning_rate * grads['b2']
        self.params['b3'] -= learning_rate * grads['b3']
        '''
        grads에서 받은 각 층의 dW,db와 학습률을 곱한값을 각 층의 W,b에서 빼주어 Loss를 변화시킨다.
       '''

In [11]:
def batchOptimization(dataset, ThreeLayerNet, learning_rate, epoch=1000):
    for i in range(epoch + 1):
        # 코드 작성
        Loss = ThreeLayerNet.forward(dataset['train_X'], dataset['one_hot_train'])
        ThreeLayerNet.gradientdescent(ThreeLayerNet.backpropagation(),learning_rate)
        if i % 10 == 0:
            train_acc = ThreeLayerNet.accuracy(dataset['train_X'], dataset['one_hot_train'])
            test_acc = ThreeLayerNet.accuracy(dataset['test_X'], dataset['one_hot_test'])
            print(i, '\t번째 Loss = ', Loss)
            print(i, '\t번째 Train_Accuracy : ', train_acc)
            print(i, '\t번째 Test_Accuracy : ', test_acc)

            train_acc_list=[]
            test_acc_list = []
            Loss_list=[]
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)
            Loss_list.append(Loss)

    return ThreeLayerNet, train_acc_list, test_acc_list, Loss_list
'''
ThreeLayerNet에 train_X[8000,784]와 one_hot_train[8000,10]을 넣어 Loss를 계산하고 Loss를 반환받는다.
ThreeLayerNet.backpropagation()를 통해 grads값을 얻어오고 ThreeLayerNet.gradientdescent를 이용해 Loss가 줄어드는 방향으로 
기계를 학습시킨다.
그것을 epoch만큼 반복하고 10번째 반복마다 train데이터와 test데이터의 정확도를 리턴받아 
Loss와 두개의 정확도를 화면에 보여준다.
rain_acc_list=[]
test_acc_list = []
Loss_list=[]
3개의 배열을 이용해 화면에 보여준 값들을 저장한다.
'''

'\nThreeLayerNet에 train_X[8000,784]와 one_hot_train[8000,10]을 넣어 Loss를 계산하고 Loss를 반환받는다.\nThreeLayerNet.backpropagation()를 통해 grads값을 얻어오고 ThreeLayerNet.gradientdescent를 이용해 Loss가 줄어드는 방향으로 \n기계를 학습시킨다.\n그것을 epoch만큼 반복하고 10번째 반복마다 train데이터와 test데이터의 정확도를 리턴받아 \nLoss와 두개의 정확도를 화면에 보여준다.\nrain_acc_list=[]\ntest_acc_list = []\nLoss_list=[]\n3개의 배열을 이용해 화면에 보여준 값들을 저장한다.\n'

In [12]:
def minibatch_Optimization(dataset, ThreeLayerNet, learning_rate, epoch=100, batch_size=100):
    np.random.seed(5)
    for i in range(epoch + 1):
        # 코드 작성

        a = np.hstack((dataset['one_hot_train'], dataset['train_X']))
        np.random.shuffle(a)

        Y_train=a[:, 0:10]
        X_train=a[:, 10:794]
        total_batch=int(a.shape[0]/batch_size)
        for k in range(total_batch) :
            x_ = X_train[batch_size * k : batch_size * (k+1), : ]
            y_ = Y_train[batch_size * k : batch_size * (k+1), : ]
            Loss = ThreeLayerNet.forward(x_,y_)
            ThreeLayerNet.gradientdescent(ThreeLayerNet.backpropagation(),learning_rate)

        Loss = ThreeLayerNet.forward(dataset['train_X'], dataset['one_hot_train'])

        if i % 10 == 0:
            train_acc = ThreeLayerNet.accuracy(dataset['train_X'], dataset['one_hot_train'])
            test_acc = ThreeLayerNet.accuracy(dataset['test_X'], dataset['one_hot_test'])
            print(i, '\t번째 Loss = ', Loss)
            print(i, '\t번째 Train_Accuracy : ', train_acc)
            print(i, '\t번째 Test_Accuracy : ', test_acc)

            train_acc_list = []
            test_acc_list = []
            Loss_list = []
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)
            Loss_list.append(Loss)

    return ThreeLayerNet, train_acc_list, test_acc_list, Loss_list
'''
shuffle가 항상 일정하게 나오게 하기 위해서 np.random.seed(5)를 해준다.
shuffle이 train_X[8000,784]와 one_hot_train[8000,10]가 묶여서 되게하기 위해서 a라는 num배열에 묶어준다 [8000,794]형태.
shuffle로 이미지의 순서들을 랜덤으로 바꿔준다.
a[8000,794] num배열을 Y_train[8000,10], X_train[8000,784] 원래의 형태로 되돌려 준다.

batch_size씩 전체 이미지를 학습시키기 위해서  batch_size의 이미지를 학습시키는것을 X_train[0]/batch_size번 반복해야한다.
batch_size씩 학습을 시키는 것을 전체 이미지에 다 시행하면 한 epoch가 끝난다.

그것을 epoch만큼 반복하고 10번째 반복마다 train데이터와 test데이터의 정확도를 리턴받아 
Loss와 두개의 정확도를 화면에 보여준다.
rain_acc_list=[]
test_acc_list = []
Loss_list=[]
3개의 배열을 이용해 화면에 보여준 값들을 저장한다



ThreeLayerNet에 train_X[8000,784]와 one_hot_train[8000,10]을 넣어 Loss를 계산하고 Loss를 반환받는다.
ThreeLayerNet.backpropagation()를 통해 grads값을 얻어오고 ThreeLayerNet.gradientdescent를 이용해 Loss가 줄어드는 방향으로 
기계를 학습시킨다.
그것을 epoch만큼 반복하고 10번째 반복마다 train데이터와 test데이터의 정확도를 리턴받아 
Loss와 두개의 정확도를 화면에 보여준다.
rain_acc_list=[]
test_acc_list = []
Loss_list=[]
3개의 배열을 이용해 화면에 보여준 값들을 저장한다.
'''

'\nshuffle가 항상 일정하게 나오게 하기 위해서 np.random.seed(5)를 해준다.\nshuffle이 train_X[8000,784]와 one_hot_train[8000,10]가 묶여서 되게하기 위해서 a라는 num배열에 묶어준다 [8000,794]형태.\nshuffle로 이미지의 순서들을 랜덤으로 바꿔준다.\na[8000,794] num배열을 Y_train[8000,10], X_train[8000,784] 원래의 형태로 되돌려 준다.\n\nbatch_size씩 전체 이미지를 학습시키기 위해서  batch_size의 이미지를 학습시키는것을 X_train[0]/batch_size번 반복해야한다.\nbatch_size씩 학습을 시키는 것을 전체 이미지에 다 시행하면 한 epoch가 끝난다.\n\n그것을 epoch만큼 반복하고 10번째 반복마다 train데이터와 test데이터의 정확도를 리턴받아 \nLoss와 두개의 정확도를 화면에 보여준다.\nrain_acc_list=[]\ntest_acc_list = []\nLoss_list=[]\n3개의 배열을 이용해 화면에 보여준 값들을 저장한다\n\n\n\nThreeLayerNet에 train_X[8000,784]와 one_hot_train[8000,10]을 넣어 Loss를 계산하고 Loss를 반환받는다.\nThreeLayerNet.backpropagation()를 통해 grads값을 얻어오고 ThreeLayerNet.gradientdescent를 이용해 Loss가 줄어드는 방향으로 \n기계를 학습시킨다.\n그것을 epoch만큼 반복하고 10번째 반복마다 train데이터와 test데이터의 정확도를 리턴받아 \nLoss와 두개의 정확도를 화면에 보여준다.\nrain_acc_list=[]\ntest_acc_list = []\nLoss_list=[]\n3개의 배열을 이용해 화면에 보여준 값들을 저장한다.\n'

In [13]:
def dropout_use_Optimizer(dataset, ThreeLayerNet, learning_rate, epoch, kill_n_h1=0.25, kill_n_h2=0.15):
    for i in range(epoch + 1):
        # 코드 작성
        L1 = ThreeLayerNet.layers['L1'].forward(dataset['train_X'])
        H1 = ThreeLayerNet.layers['SiLU1'].forward(L1)
        U1 = np.random.rand(*H1.shape) < (1-kill_n_h1)
        H1 *= U1
        L2 = ThreeLayerNet.layers['L2'].forward(H1)
        H2 = ThreeLayerNet.layers['SiLU2'].forward(L2)
        U2 = np.random.rand(*H2.shape) < (1-kill_n_h2)
        H2 *= U2

        L3 = ThreeLayerNet.layers['L3'].forward(H2)
        Loss = ThreeLayerNet.lastLayer.forward(L3,dataset['one_hot_train'])

        x = ThreeLayerNet.lastLayer.backward()
        x = ThreeLayerNet.layers['L3'].backward(x)
        x *= U2
        x = ThreeLayerNet.layers['SiLU2'].backward(x)
        x = ThreeLayerNet.layers['L2'].backward(x)
        x *= U1
        x = ThreeLayerNet.layers['SiLU1'].backward(x)
        x = ThreeLayerNet.layers['L1'].backward(x)


        grads = {}
        grads['W1'] = ThreeLayerNet.layers['L1'].dW
        grads['b1'] = ThreeLayerNet.layers['L1'].db
        grads['W2'] = ThreeLayerNet.layers['L2'].dW
        grads['b2'] = ThreeLayerNet.layers['L2'].db
        grads['W3'] = ThreeLayerNet.layers['L3'].dW
        grads['b3'] = ThreeLayerNet.layers['L3'].db

        ThreeLayerNet.gradientdescent(grads, learning_rate)

        if i % 10 == 0:
            train_acc = ThreeLayerNet.accuracy(dataset['train_X'], dataset['one_hot_train'])
            test_acc = ThreeLayerNet.accuracy(dataset['test_X'], dataset['one_hot_test'])
            print(i, '\t번째 Loss = ', Loss)
            print(i, '\t번째 Train_Accuracy : ', train_acc)
            print(i, '\t번째 Test_Accuracy : ', test_acc)

            train_acc_list = []
            test_acc_list = []
            Loss_list = []
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)
            Loss_list.append(Loss)
    return ThreeLayerNet, train_acc_list, test_acc_list, Loss_list
'''
batchOptimization()함수와 비슷하다 차이점이 있다면  H1, H2의 forward에서 값을 리턴하기 전에
H1, H2를 kill_n_h1, kill_n_h2만큼 값을 죽이는 것이다. 
train시에는 kill_n_h1, kill_n_h2만큼 0을 가지는 U1,U2 num배열 H1, H2 리턴값에 곱해준다.
test시에는  kill_n_h1, kill_n_h2값을 H1, H2 리턴값에 직접 곱해준다. 

따라서 로컬 gradient를 구할때도 H1 *= U1, H2 *= U2 연산에 대한 backward도 추가하여 준다.

위에서 구현한 함수를 ThreeLayer에 forward와 backward함수를 이용하여도 가능하겠지만 수정이 어려워 사이사이에 
dropout기능만 추가하도록 다시 구현하였습니다.
'''

'\nbatchOptimization()함수와 비슷하다 차이점이 있다면  H1, H2의 forward에서 값을 리턴하기 전에\nH1, H2를 kill_n_h1, kill_n_h2만큼 값을 죽이는 것이다. \ntrain시에는 kill_n_h1, kill_n_h2만큼 0을 가지는 U1,U2 num배열 H1, H2 리턴값에 곱해준다.\ntest시에는  kill_n_h1, kill_n_h2값을 H1, H2 리턴값에 직접 곱해준다. \n\n따라서 로컬 gradient를 구할때도 H1 *= U1, H2 *= U2 연산에 대한 backward도 추가하여 준다.\n\n위에서 구현한 함수를 ThreeLayer에 forward와 backward함수를 이용하여도 가능하겠지만 수정이 어려워 사이사이에 \ndropout기능만 추가하도록 다시 구현하였습니다.\n'

In [14]:
# 과제 채점을 위한 세팅
train_X, train_label, test_X, test_label = train_test_split(mnist)


one_hot_train = one_hot_encoding(train_label)
one_hot_test = one_hot_encoding(test_label)

dataset = {}
dataset['train_X'] = train_X
dataset['test_X'] = test_X
dataset['one_hot_train'] = one_hot_train
dataset['one_hot_test'] = one_hot_test

neournlist = [784, 60, 30, 10]

TNN_batchOptimizer = ThreeLayerNet(neournlist)
TNN_minibatchOptimizer = copy.deepcopy(TNN_batchOptimizer)
TNN_dropOut = copy.deepcopy(TNN_minibatchOptimizer)

In [15]:
# 채점은 이 것의 결과값으로 할 예정입니다.

trained_batch, tb_train_acc_list, tb_test_acc_list, tb_loss_list = batchOptimization(dataset, TNN_batchOptimizer, 0.1, 1000)

trained_minibatch, tmb_train_acc_list, tmb_test_acc_list, tb_loss_list = minibatch_Optimization(dataset,
                                                                                                TNN_minibatchOptimizer,
                                                                                                0.1, epoch=100,
                                                                                                batch_size=100)

trained_dropout, td_train_acc_list, td_test_acc_list, td_loss_list = dropout_use_Optimizer(dataset, TNN_dropOut,
                                                                                           0.1, 1000, 0.25,
                                                                                           0.15)


0 	번째 Loss =  2.34547452273538
0 	번째 Train_Accuracy :  0.116375
0 	번째 Test_Accuracy :  0.106
10 	번째 Loss =  2.0206773159218954
10 	번째 Train_Accuracy :  0.368
10 	번째 Test_Accuracy :  0.343
20 	번째 Loss =  1.6611898927508257
20 	번째 Train_Accuracy :  0.524875
20 	번째 Test_Accuracy :  0.5455
30 	번째 Loss =  1.2473926930255774
30 	번째 Train_Accuracy :  0.686875
30 	번째 Test_Accuracy :  0.7195
40 	번째 Loss =  0.9064624954059618
40 	번째 Train_Accuracy :  0.774375
40 	번째 Test_Accuracy :  0.806
50 	번째 Loss =  0.7156998896231759
50 	번째 Train_Accuracy :  0.816
50 	번째 Test_Accuracy :  0.848
60 	번째 Loss =  0.6064104766120427
60 	번째 Train_Accuracy :  0.8435
60 	번째 Test_Accuracy :  0.8715
70 	번째 Loss =  0.5360575732854721
70 	번째 Train_Accuracy :  0.857375
70 	번째 Test_Accuracy :  0.885
80 	번째 Loss =  0.48662669243910334
80 	번째 Train_Accuracy :  0.868
80 	번째 Test_Accuracy :  0.8905
90 	번째 Loss =  0.449857730237532
90 	번째 Train_Accuracy :  0.877125
90 	번째 Test_Accuracy :  0.897
100 	번째 Loss =  0.46515901429388

820 	번째 Loss =  0.13433378947165428
820 	번째 Train_Accuracy :  0.963625
820 	번째 Test_Accuracy :  0.941
830 	번째 Loss =  0.1327783991296047
830 	번째 Train_Accuracy :  0.9635
830 	번째 Test_Accuracy :  0.941
840 	번째 Loss =  0.1312395822653643
840 	번째 Train_Accuracy :  0.964375
840 	번째 Test_Accuracy :  0.942
850 	번째 Loss =  0.1297171502061193
850 	번째 Train_Accuracy :  0.965
850 	번째 Test_Accuracy :  0.9425
860 	번째 Loss =  0.12821091742894056
860 	번째 Train_Accuracy :  0.965625
860 	번째 Test_Accuracy :  0.943
870 	번째 Loss =  0.1267207011794011
870 	번째 Train_Accuracy :  0.965625
870 	번째 Test_Accuracy :  0.943
880 	번째 Loss =  0.12524632123271082
880 	번째 Train_Accuracy :  0.966
880 	번째 Test_Accuracy :  0.943
890 	번째 Loss =  0.12378759978846161
890 	번째 Train_Accuracy :  0.966375
890 	번째 Test_Accuracy :  0.943
900 	번째 Loss =  0.12234436148816207
900 	번째 Train_Accuracy :  0.96675
900 	번째 Test_Accuracy :  0.943
910 	번째 Loss =  0.12091643354296018
910 	번째 Train_Accuracy :  0.967125
910 	번째 Test_Accuracy :

KeyboardInterrupt: 